In [1]:
import pandas as pd
import pdfplumber
import requests
import urllib
from bs4 import BeautifulSoup
import nltk
import re
import os

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Acer\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
carac_especial = ['(',')','-',',',', ',' ,','.','@',':','*','☎','●','_','n.º','º','–','nº','“','”', '%', '$','R','/','!','+','✔',
                  'A','O','Á','à','À','____','3ª','&','➧','Nº','?','1ª','','|','e','Ao', '\r','e,', '(e', '*.\r', 'É', 'J.',
                  'S/A.', 'S/A', '\t','\xa0','(DVD)', '(cd)']
all_stopwords = stopwords.words('portuguese')
char = all_stopwords
char.extend(carac_especial)
char.extend([f'{i}' for i in range(1000)])

nomes = ['esquizofrenia', 'esquizofrenia,', 'esquizofrenia.', 'esquizofrenia, ', 'esquizofrenia. ', 
         ' esquizofrenia', 'esquizofrenia ', 'esquizofrenia.\r', '"esquizofrenia"', 'esquizofrenia!', 
         'esquizofrênico','esquizofrênico,', 'esquizofrênico, ', 'esquizofrênico.', 'esquizofrênico. ',
         ', esquizofrênico']

In [3]:
def limpa_texto(link, especiais, nomes, tamanho):
    
    html = urllib.request.urlopen(link).read()
    soup = BeautifulSoup(html)

    for script in soup(["script", "style"]):
        script.decompose()
        
    lista = []

    for i in soup.stripped_strings:
        lista.append(i)
        
    miolo = []
    for i in lista:
        if len(i) > tamanho:
            miolo.append(i.replace('\r\n',''))
    
    if miolo != []: 
        miolo.pop() # dropar o Copyright
    
    texto = (' '.join(miolo)).lower()

#     titulo = soup.title.text.split(' ')
#     especiais.extend(titulo)
    
    lista_palavras = texto.split(' ')
    lista_palavras_sem_especiais = [word for word in lista_palavras if not word in especiais]

    r = re.compile(".*\r|\d+|.*@|.*#|.\\t|.*\\xa0|.*\\'")
    especiais2 = list(filter(r.match, lista_palavras_sem_especiais))
    
    if 'esquizofrenia.\r' in especiais2:
        especiais2.remove('esquizofrenia.\r')

    lista_palavras_sem_especiais2 = [word for word in lista_palavras_sem_especiais if not word in especiais2]

    limpo = ' '.join(lista_palavras_sem_especiais2)
    
    indices = []

    for palavra in range(len(lista_palavras_sem_especiais2)):
        if lista_palavras_sem_especiais2[palavra] in nomes:
            print(palavra)
            indices.append(palavra)

#     text_file = open("texto.txt", "wt")
#     n = text_file.write(limpo)
#     text_file.close()
    
    return limpo, lista_palavras_sem_especiais2, indices

In [4]:
link = 'https://www1.folha.uol.com.br/fsp/ilustrad/fq1001200106.htm'

In [5]:
# texto, lista, indices = limpa_texto(link, char, nomes, 90)

In [6]:
# texto

In [7]:
def salva_trechos(indices, lista, pasta, n_contexto):
    for j in range(len(indices)):
        trecho = []
        
        try:
            for i in range(indices[j] - n_contexto, indices[j] + n_contexto):
                trecho.append(lista[i])
                texto = ' '.join(trecho)
                text_file = open('data/' + pasta + f'/trecho{j}.txt', 'wt')
                n = text_file.write(texto)
                text_file.close()
        except: 
            print('Erro no índice anterior a palavra.')
            try:
                for i in range(indices[j], indices[j] + n_contexto):
                    trecho.append(lista[i])
                    texto = ' '.join(trecho)
                    text_file = open('data/' + pasta + f'/trecho{j}.txt', 'wt')
                    n = text_file.write(texto)
                    text_file.close()
            except:
                print('Erro no índice posterior a palavra.')
        else: 
            for i in range(indices[j] - n_contexto, indices[j]):
                trecho.append(lista[i])
                texto = ' '.join(trecho)
                text_file = open('data/' + pasta + f'/trecho{j}.txt', 'wt')
                n = text_file.write(texto)
                text_file.close()

In [8]:
def gerador_database(lista_link, n_contexto, tamanho):
    for i in range(len(lista_link)):
        
        os.makedirs(f'data/pasta{i+2}',exist_ok=True)
        _, lista, indices = limpa_texto(lista_link[i], char, nomes, tamanho)
        salva_trechos(indices, lista, f"pasta{i+2}", n_contexto)     

In [9]:
lista_link = [
    'https://www1.folha.uol.com.br/fsp/ilustrad/fq0401200121.htm',
    'https://www1.folha.uol.com.br/folha/mundo/ult94u16986.shtml?origin=folha',
    'https://www1.folha.uol.com.br/fsp/ilustrad/fq1001200106.htm',
    'https://www1.folha.uol.com.br/folha/ciencia/ult306u2066.shtml?origin=folha',
    'https://www1.folha.uol.com.br/folha/ciencia/ult306u2071.shtml?origin=folha',
    'https://www1.folha.uol.com.br/folha/ilustrada/ult90u9192.shtml?origin=folha',
    'https://www1.folha.uol.com.br/folha/ciencia/ult306u2087.shtml?origin=folha',
    'https://www1.folha.uol.com.br/fsp/ciencia/fe1801200107.htm',
    'https://www1.folha.uol.com.br/folha/ciencia/ult306u2418.shtml?origin=folha',
    'https://www1.folha.uol.com.br/fsp/equilibrio/eq0802200113.htm',
    'https://www1.folha.uol.com.br/folha/ciencia/ult306u2546.shtml?origin=folha',
    'https://www1.folha.uol.com.br/folha/ilustrada/ult90u10685.shtml?origin=folha',
    'https://www1.folha.uol.com.br/fsp/folhatee/fm1902200116.htm',
    'https://www1.folha.uol.com.br/fsp/folhatee/fm1902200117.htm',
    'https://www1.folha.uol.com.br/folha/ciencia/ult306u2709.shtml?origin=folha',
    'https://www1.folha.uol.com.br/fsp/ciencia/fe1403200101.htm',
    'https://www1.folha.uol.com.br/folha/ciencia/ult306u3019.shtml?origin=folha',
    'https://www1.folha.uol.com.br/fsp/ciencia/fe2103200105.htm',
    'https://www1.folha.uol.com.br/fsp/mais/fs2503200110.htm',
    'http://www1.folha.uol.com.br/fsp/ilustrad/fq2903200109.htm',
    'https://www1.folha.uol.com.br/fsp/ilustrad/fq1105200123.htm',
    'http://www1.folha.uol.com.br/fsp/brasil/fc0204200112.htm',
    'https://www1.folha.uol.com.br/fsp/ilustrad/fq3105200111.htm',
    'https://www1.folha.uol.com.br/fsp/folhatee/fm0207200104.htm',
    'https://www1.folha.uol.com.br/fsp/ilustrad/fq0207200113.htm',
    'https://www1.folha.uol.com.br/folha/cotidiano/ult95u33066.shtml',
    'https://www1.folha.uol.com.br/fsp/ilustrad/fq0408200119.htm',
    'https://www1.folha.uol.com.br/fsp/ilustrad/fq1708200129.htm',
    'https://www1.folha.uol.com.br/fsp/acontece/ac0909200103.htm',
    'https://www1.folha.uol.com.br/fsp/mais/fs0909200117.htm',
    'https://www1.folha.uol.com.br/fsp/opiniao/fz2109200109.htm',
    'https://www1.folha.uol.com.br/fsp/opiniao/fz0110200110.htm',
    'https://www1.folha.uol.com.br/fsp/opiniao/fz3010200111.htm',
    'https://www1.folha.uol.com.br/fsp/opiniao/fz0111200111.htm',
    'https://www1.folha.uol.com.br/fsp/mais/fs0411200112.htm',
    'https://www1.folha.uol.com.br/fsp/ilustrad/fq0203200103.htm',
    'https://www1.folha.uol.com.br/fsp/ombudsma/om1803200101.htm'
]

In [10]:
# gerador_database(lista_link, 5, 70)

In [11]:
lista_link2 = [
    "https://www1.folha.uol.com.br/fsp/ilustrad/fq0401200121.htm",
"https://www1.folha.uol.com.br/folha/mundo/ult94u16986.shtml?origin=folha",
"https://www1.folha.uol.com.br/fsp/ilustrad/fq1001200106.htm",
"https://www1.folha.uol.com.br/folha/ciencia/ult306u2066.shtml?origin=folha",
"https://www1.folha.uol.com.br/folha/ciencia/ult306u2071.shtml?origin=folha",
"https://www1.folha.uol.com.br/folha/ilustrada/ult90u9192.shtml?origin=folha",
"https://www1.folha.uol.com.br/folha/ciencia/ult306u2087.shtml?origin=folha",
"https://www1.folha.uol.com.br/fsp/ciencia/fe1801200107.htm",
"https://www1.folha.uol.com.br/folha/ciencia/ult306u2418.shtml?origin=folha",
"https://www1.folha.uol.com.br/fsp/equilibrio/eq0802200113.htm",
"https://www1.folha.uol.com.br/folha/ciencia/ult306u2546.shtml?origin=folha",
"https://www1.folha.uol.com.br/folha/ilustrada/ult90u10685.shtml?origin=folha",
"https://www1.folha.uol.com.br/fsp/folhatee/fm1902200116.htm",
"https://www1.folha.uol.com.br/fsp/folhatee/fm1902200117.htm",
"https://www1.folha.uol.com.br/folha/ciencia/ult306u2709.shtml?origin=folha",
"https://www1.folha.uol.com.br/fsp/ciencia/fe1403200101.htm",
"https://www1.folha.uol.com.br/folha/ciencia/ult306u3019.shtml?origin=folha",
"https://www1.folha.uol.com.br/fsp/ciencia/fe2103200105.htm",
"https://www1.folha.uol.com.br/fsp/mais/fs2503200110.htm",
"http://www1.folha.uol.com.br/fsp/ilustrad/fq2903200109.htm",
"http://www1.folha.uol.com.br/fsp/ilustrad/fq0704200102.htm",
"http://www1.folha.uol.com.br/fsp/mais/fs0804200111.htm",
"http://www1.folha.uol.com.br/folha/ciencia/ult306u3264.shtml",
"http://www1.folha.uol.com.br/fsp/ciencia/fe1104200101.htm",
"http://www1.folha.uol.com.br/folha/ciencia/ult306u3428.shtml",
"https://www1.folha.uol.com.br/fsp/mundo/ft1105200113.htm",
"https://www1.folha.uol.com.br/fsp/ciencia/fe1305200108.htm",
"https://www1.folha.uol.com.br/fsp/ciencia/fe1305200104.htm",
"https://www1.folha.uol.com.br/fsp/mundo/ft1705200108.htm",
"https://www1.folha.uol.com.br/folha/reuters/ult112u2519.shtml",
"https://www1.folha.uol.com.br/fsp/equilibrio/eq0507200113.htm",
"https://www1.folha.uol.com.br/folha/equilibrio/noticias/ult263u194.shtml",
"https://www1.folha.uol.com.br/fsp/equilibrio/eq1207200115.htm",
"https://www1.folha.uol.com.br/folha/reuters/ult112u3660.shtml",
"https://www1.folha.uol.com.br/fsp/equilibrio/eq2607200103.htm",
"https://www1.folha.uol.com.br/folha/equilibrio/noticias/ult263u240.shtml",
"https://www1.folha.uol.com.br/folha/reuters/ult112u4288.shtml",
"https://www1.folha.uol.com.br/fsp/cotidian/ff1208200128.htm",
"https://www1.folha.uol.com.br/fsp/cotidian/ff1208200127.htm",
"https://www1.folha.uol.com.br/fsp/cotidian/ff1208200126.htm",
"https://www1.folha.uol.com.br/folha/equilibrio/noticias/ult263u300.shtml",
"https://www1.folha.uol.com.br/folha/equilibrio/noticias/ult263u302.shtml",
"https://www1.folha.uol.com.br/folha/ciencia/ult306u4592.shtml",
"https://www1.folha.uol.com.br/folha/reuters/ult112u4627.shtml",
"https://www1.folha.uol.com.br/fsp/ciencia/fe1508200104.htm",
"https://www1.folha.uol.com.br/folha/ciencia/ult306u4618.shtml",
"https://www1.folha.uol.com.br/folha/ciencia/ult306u4930.shtml",
"https://www1.folha.uol.com.br/fsp/equilibrio/eq1810200106.htm",
"https://www1.folha.uol.com.br/folha/bbc/ult272u6836.shtml",
"https://www1.folha.uol.com.br/fsp/ciencia/fe0511200101.htm",
"https://www1.folha.uol.com.br/fsp/ilustrad/fq0711200113.htm",
"https://www1.folha.uol.com.br/folha/ciencia/ult306u5294.shtml",
"https://www1.folha.uol.com.br/fsp/ilustrad/fq1711200121.htm",
"https://www1.folha.uol.com.br/folha/ilustrada/ult90u19406.shtml",
"https://www1.folha.uol.com.br/folha/mundo/ult94u34454.shtml",
"https://www1.folha.uol.com.br/fsp/ciencia/fe1012200101.htm",
"https://www1.folha.uol.com.br/fsp/opiniao/fz1212200110.htm",
"https://www1.folha.uol.com.br/folha/equilibrio/noticias/ult263u691.shtml",
"https://www1.folha.uol.com.br/fsp/cotidian/ff3012200118.htm",
"https://www1.folha.uol.com.br/folha/reuters/ult112u10188.shtml",
"https://www1.folha.uol.com.br/folha/reuters/ult112u10191.shtml",
"https://www1.folha.uol.com.br/fsp/ilustrad/fq1105200123.htm",
"http://www1.folha.uol.com.br/fsp/brasil/fc0204200112.htm",
"https://www1.folha.uol.com.br/fsp/ilustrad/fq3105200111.htm",
"https://www1.folha.uol.com.br/fsp/folhatee/fm0207200104.htm",
"https://www1.folha.uol.com.br/fsp/ilustrad/fq0207200113.htm",
"https://www1.folha.uol.com.br/folha/cotidiano/ult95u33066.shtml",
"https://www1.folha.uol.com.br/fsp/ilustrad/fq0408200119.htm",
"https://www1.folha.uol.com.br/fsp/ilustrad/fq1708200129.htm",
"https://www1.folha.uol.com.br/fsp/acontece/ac0909200103.htm",
"https://www1.folha.uol.com.br/fsp/mais/fs0909200117.htm",
"https://www1.folha.uol.com.br/fsp/opiniao/fz2109200109.htm",
"https://www1.folha.uol.com.br/fsp/opiniao/fz0110200110.htm",
"https://www1.folha.uol.com.br/fsp/opiniao/fz3010200111.htm",
"https://www1.folha.uol.com.br/fsp/opiniao/fz0111200111.htm",
"https://www1.folha.uol.com.br/fsp/mais/fs0411200112.htm",
"https://www1.folha.uol.com.br/fsp/ilustrad/fq0203200103.htm",
"https://www1.folha.uol.com.br/fsp/ombudsma/om1803200101.htm",
"https://www1.folha.uol.com.br/fsp/opiniao/fz1501200210.htm",
"https://www1.folha.uol.com.br/fsp/mundo/ft0401200215.htm",
"https://www1.folha.uol.com.br/fsp/ombudsma/om1702200202.htm",
"https://www1.folha.uol.com.br/fsp/cotidian/ff1702200217.htm",
"https://www1.folha.uol.com.br/fsp/ilustrad/fq2702200213.htm",
"http://www1.folha.uol.com.br/fsp/ilustrad/fq3003200203.htm",
"http://www1.folha.uol.com.br/fsp/cotidian/ff0904200204.htm",
"http://www1.folha.uol.com.br/fsp/opiniao/fz2504200210.htm",
"http://www1.folha.uol.com.br/fsp/ilustrad/fq0405200213.htm",
"http://www1.folha.uol.com.br/fsp/opiniao/fz0806200201.htm",
"http://www1.folha.uol.com.br/folha/pensata/ult510u63.shtml",
"http://www1.folha.uol.com.br/fsp/opiniao/fz2406200211.htm",
"http://www1.folha.uol.com.br/fsp/ilustrad/fq1907200228.htm",
"http://www1.folha.uol.com.br/fsp/opiniao/fz2807200201.htm",
"http://www1.folha.uol.com.br/fsp/mais/fs1808200210.htm",
"http://www1.folha.uol.com.br/fsp/dinheiro/fi1109200207.htm",
"http://www1.folha.uol.com.br/fsp/opiniao/fz0604200304.htm",
"http://www1.folha.uol.com.br/fsp/ilustrad/fq0504200319.htm",
"http://www1.folha.uol.com.br/fsp/opiniao/fz1304200310.htm",
"http://www1.folha.uol.com.br/fsp/ilustrad/fq2512200211.htm",
"http://www1.folha.uol.com.br/fsp/brasil/fc0901200313.htm",
"http://www1.folha.uol.com.br/fsp/opiniao/fz0901200304.htm",
"http://www1.folha.uol.com.br/fsp/brasil/fc2601200330.htm",
"http://www1.folha.uol.com.br/fsp/opiniao/fz0604200304.htm",
"http://www1.folha.uol.com.br/fsp/ilustrad/fq0504200319.htm",
"http://www1.folha.uol.com.br/fsp/opiniao/fz1304200310.htm",
"http://www1.folha.uol.com.br/fsp/ilustrad/fq2204200309.htm",
"http://www1.folha.uol.com.br/fsp/opiniao/fz1909200207.htm",
"http://www1.folha.uol.com.br/folha/educacao/ult305u10765.shtml",
"http://www1.folha.uol.com.br/fsp/folhatee/fm2810200209.htm",
"http://www1.folha.uol.com.br/fsp/ilustrad/fq2512200211.htm",
"http://www1.folha.uol.com.br/fsp/brasil/fc0901200313.htm",
"http://www1.folha.uol.com.br/fsp/opiniao/fz0901200304.htm",
"http://www1.folha.uol.com.br/fsp/brasil/fc2601200330.htm",
"http://www1.folha.uol.com.br/fsp/opiniao/fz0604200304.htm",
"http://www1.folha.uol.com.br/fsp/ilustrad/fq0504200319.htm",
"http://www1.folha.uol.com.br/fsp/opiniao/fz1304200310.htm",
"http://www1.folha.uol.com.br/fsp/ilustrad/fq2204200309.htm",
"https://www1.folha.uol.com.br/fsp/esporte/fk2005200314.htm",
"https://www1.folha.uol.com.br/fsp/mais/fs0106200305.htm",
"http://www1.folha.uol.com.br/fsp/mais/fs0106200311.htm",
"https://www1.folha.uol.com.br/fsp/equilibrio/eq0506200312.htm",
"https://www1.folha.uol.com.br/folha/ilustrada/ult90u33864.shtml",
"https://www1.folha.uol.com.br/fsp/opiniao/fz1607200304.htm",
"https://www1.folha.uol.com.br/fsp/mais/fs2007200310.htm",
"https://www1.folha.uol.com.br/folha/dinheiro/ult91u70666.shtml",
"https://www1.folha.uol.com.br/folha/brasil/ult96u52200.shtml",
"https://www1.folha.uol.com.br/fsp/opiniao/fz0508200309.htm",
"https://www1.folha.uol.com.br/fsp/brasil/fc1808200317.htm",
"https://www1.folha.uol.com.br/fsp/ciencia/fe3108200304.htm"
]

In [12]:
gerador_database(lista_link2, 10, 70)

137
102
33
8
17
45
95
102
152
8
18
42
126
133
180
213
35
43
122
7
45
124
27
342
9
21
180
35
3
7
14
45
117
129
389
7
10
34
57
105
158
207
7
11
645
81
397
10
670
6
49
64
84
118
5
33
71
114
156
177
198
246
266
313
7
45
143
173
97
239
63
7
21
165
212
543
530
167
5
12
55
154
7
15
38
147
176
Erro no índice anterior a palavra.
Erro no índice posterior a palavra.
8
16
39
148
177
92
34
4
38
47
211
216
3
210
215
43
5
16
41
68
130
8
18
55
64
28
17
105
161
252
330
28
62
11
71
6
26
144
113
9
12
23
11
70
83
214
245
7
13
51
62
221
13
231
284
99
92
120
92
348
1030
53
14
48
89
90
601
184
82
379
254
368
320
241
Erro no índice anterior a palavra.
Erro no índice posterior a palavra.
285
403
249
178
344
214
106
761
240
150
80
74
9
96
57
150
80
74
15
226
45
9
96
57
150
80
74
15
226
1040
391
255
12
102
1193
243
745
187


In [13]:
# url = 'https://www1.folha.uol.com.br/folha/pensata/helioschwartsman/ult510u356290.shtml'
url = 'https://www1.folha.uol.com.br/fsp/opiniao/fz2406200211.htm'

texto, lista, indices = limpa_texto(url, char, nomes, 90)

344


In [14]:
texto

'"a política alianças pt, próprio radicais, saltou extremo outro. pt, sempre avesso alianças, criticou oito anos união psdb pfl. aliança construída tendo base amplo programa reformas sociais econômicas. alianças pt (orestes) quércia (pmdb), petistas chamaram bonito, pl edir macedo, exclusivamente conveniência eleitoral constituem, agora, atitude desavergonhada desta campanha." "é doer onda denúncias consistência contra pt. vale tudo tirar crença ético maior nível moral dentre partidos país. brasileiros, certeza, acham partidos devem ser investigados, sim. dessa forma. trás maioria dessas denúncias, adversários pt. sabem próprios fizessem denúncias cairiam descrença. utilizam-se grande parte imprensa jogo sujo." "o atual governador mineiro (itamar franco) poderia estar bem eleitoralmente, dificuldade ser franco. flertou frente trabalhista gestação acenou tucanos. agora, enquanto, encerra-se alianças pt nível estadual, apóia aécio neves, psdb. personagem corre risco ficar apoiado calcanh

In [15]:
len(lista_link2)

128